In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from spellchecker import SpellChecker
import string 


In [ ]:
df = pd.read_csv("Coronavirus_dec05.csv")
df.isnull().values.any()
df.shape

In [ ]:
df_duplicates_removed = pd.DataFrame.drop_duplicates(df)

df_duplicates_removed.head()
df.shape

In [ ]:
df = df.rename(columns = { '3': 'Tweet'}, inplace = False)
df = df[["Tweet"]]
df.shape

In [ ]:
df.head()

In [ ]:
#sentimenet lebeling
sid = SentimentIntensityAnalyzer()

def sentiment_score(tweet):
    # Calling the polarity_scores method on sid and passing in the text outputs a dictionary with negative, neutral, positive, and compound scores for the input text
    text = "'''" + str(tweet) + "'''"
    scores=sid.polarity_scores(text)   
    return sentiment_lebel(scores["compound"])

def sentiment_lebel(score):
    #-1 to -0.5 , -0.5 to 0 ,   0 to 0.5  ,   0.5 to  1
    if score < -0.5 and score >= -1:
     return "Storngly Nagative"
    elif score < 0 and score >= -0.5 :
     return "Nagative"
    if score==0:
     return "Neutral"
    elif score >0 and score <= 0.5:
     return "Positive"
    elif score > 0.5 and score <= 1:
     return "Storngly Positive"
    else:
     return "NAN"



# function to convert numbers to words
from num2words import num2words

def num_to_words(text):
    # splitting text into words with space
    after_spliting = text.split()

    for index in range(len(after_spliting)):
        if after_spliting[index].isdigit():
            after_spliting[index] = num2words(after_spliting[index])

    # joining list into string with space
    numbers_to_words = ' '.join(after_spliting)
    return numbers_to_words

# Remove punctuations

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(tweet):
    return tweet.translate(str.maketrans('', '', PUNCT_TO_REMOVE))



#Remove stopwords 



STOPWORDS = set(stopwords.words('english'))
STOPWORDS.remove("not")
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

#stemming
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

# spelling correction using spellchecker

spell_corrector = SpellChecker()

def spell_correction(text):
    # initialize empty list to save correct spell words
    correct_words = []
    # extract spelling incorrect words by using unknown function of spellchecker
    misSpelled_words = spell_corrector.unknown(text.split())

    for each_word in text.split():
        if each_word in misSpelled_words:
            right_word = spell_corrector.correction(each_word)
            correct_words.append(right_word)
        else:
            correct_words.append(each_word)

    # joining correct_words list into single string
    correct_spelling = ' '.join(correct_words)
    return correct_spelling

def preprocess_text(sen):
    
    #Noise Removal
    
    #removing urls
    sentence = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', sen)
    
    
    # Removing html tags
    sentence = remove_tags(sentence)
    
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    # Convert numbers to words
    sentence = num_to_words(sentence)
    
    
    # Remove punctuations
    sentence =remove_punctuation(sentence)

     # Remove stopwords
    sentence =remove_stopwords(sentence)
    
    
    #stemming 
    sentence =stem_words(sentence)
    
     # Removing RT 
    sentence=re.sub(r'RT','',sentence)
    
    #spell check
    #sentence = spell_correction(sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

X = []
y = []
sentences = list(df['Tweet'])
for sen in sentences:
    sentence=preprocess_text(sen)
    y.append(sentiment_score(sentence))
    sentiment_score(sen)
    X.append(preprocess_text(sen))
    

my_df = {'Tweet': X,
         'sentiment':y 
       } 

df = pd.DataFrame(my_df) 

In [ ]:
#saving df to csv
df.to_csv('clean_data_dec0.csv', index=False) 

In [ ]:
df.head()